In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
!nvidia-smi

Mon Mar 29 12:46:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import pandas as pd
import numpy as np

import urllib3
import zipfile
import shutil
import os
import random
import re
import unicodedata

In [4]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input, LSTM, Embedding, Dense, Masking

In [5]:
import tensorflow as tf

In [6]:
from google.colab import drive

In [7]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## 1. Data Load

In [7]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/Natural_Language_Processing/data/kor-eng.zip

Archive:  /content/drive/My Drive/Colab Notebooks/Natural_Language_Processing/data/kor-eng.zip
  inflating: _about.txt              
  inflating: kor.txt                 


In [8]:
lines = pd.read_csv('kor.txt', names = ['src' , 'tar'],
                    sep = '\t',
                    index_col = False)

In [9]:
len(lines)

3648

In [10]:
lines.head()

,src,tar
0,Go.,가.
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?


In [11]:
lines.tar = lines.tar.apply(lambda x : '\t' + x + '\n')
# 시작심볼, 종료심볼 추가

In [12]:
lines.sample(10)

,src,tar
3112,I assure you Tom will be perfectly safe.,\t톰은 완전 멀쩡할 거라니까.\n
3194,I just don't want anyone to be mad at me.,\t나는 그 누구도 나한테 화를 안냈으면 좋겠을 뿐이야.\n
1393,What's your cat's name?,\t네 고양이의 이름은 뭐야?\n
545,Tom confessed.,\t톰이 자백했어.\n
2785,"Tom is going to be there, isn't he?",\t톰은 거기에 가 있을 거지?\n
2185,Why are you acting so stupid?,\t넌 왜 그렇게 바보같이 굴어?\n
2976,I was pretty young back in those days.,\t그 때는 꽤 어렸었는데.\n
1404,Can we still be friends?,\t우린 아직도 친구일 수 있을까?\n
397,Examine this.,\t이걸 조사해봐.\n
2821,"I caught a cold, and I have a fever.",\t감기에 걸려서 열이 나.\n


## 1. 글자단위 번역기

### 1) Data Preprocessing

In [13]:
# 글자 집합 구축

src_vocab = set()
for line in lines.src :
  for char in line :
    src_vocab.add(char)

tar_vocab = set()
for line in lines.tar :
  for char in line :
    tar_vocab.add(char)

In [14]:
# 글자 집합의 크기 (영어, 한국어)
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1

src_vocab_size , tar_vocab_size

(75, 912)

In [15]:
# 정렬하여 순서를 정해준 뒤 인덱스로 내용 출력
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

print(src_vocab[45:75] , '\n' , tar_vocab[45:75])

['Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '°', 'ï'] 
 ['간', '갇', '갈', '감', '갑', '값', '갔', '강', '갖', '같', '개', '객', '갰', '걀', '걔', '거', '걱', '건', '걷', '걸', '검', '겁', '것', '게', '겐', '겠', '겨', '격', '겪', '견']


- 각 글자에 인덱스 부여

In [16]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])

print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, "'": 6, ',': 7, '-': 8, '.': 9, '0': 10, '1': 11, '2': 12, '3': 13, '4': 14, '5': 15, '6': 16, '7': 17, '8': 18, '9': 19, ':': 20, ';': 21, '?': 22, 'A': 23, 'B': 24, 'C': 25, 'D': 26, 'E': 27, 'F': 28, 'G': 29, 'H': 30, 'I': 31, 'J': 32, 'K': 33, 'L': 34, 'M': 35, 'N': 36, 'O': 37, 'P': 38, 'Q': 39, 'R': 40, 'S': 41, 'T': 42, 'U': 43, 'V': 44, 'W': 45, 'Y': 46, 'a': 47, 'b': 48, 'c': 49, 'd': 50, 'e': 51, 'f': 52, 'g': 53, 'h': 54, 'i': 55, 'j': 56, 'k': 57, 'l': 58, 'm': 59, 'n': 60, 'o': 61, 'p': 62, 'q': 63, 'r': 64, 's': 65, 't': 66, 'u': 67, 'v': 68, 'w': 69, 'x': 70, 'y': 71, 'z': 72, '°': 73, 'ï': 74}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '%': 6, '(': 7, ')': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'H': 29, 'M': 30, 'N': 31, 'T': 32, 'a': 33, 'd': 34, 'h': 35, 'i': 36, 'm': 37, 'o': 38, 'p': 3

- 정수 인코딩

In [17]:
# 영어문장에 대한 정수인코딩
encoder_input = []

for line in lines.src:
  temp_X = []
  for w in line :
    temp_X.append(src_to_index[w])
  encoder_input.append(temp_X)

In [18]:
print(encoder_input[:5])

[[29, 61, 9], [30, 55, 9], [40, 67, 60, 2], [40, 67, 60, 9], [45, 54, 61, 22]]


In [19]:
# 한국어 데이터에 대한 정수인코딩
decoder_input = []

for line in lines.tar:
  temp_X = []
  for w in line:
    temp_X.append(tar_to_index[w])
  decoder_input.append(temp_X)

In [20]:
print(decoder_input[:5])

[[1, 44, 11, 2], [1, 548, 195, 11, 2], [1, 288, 570, 4, 2], [1, 288, 570, 11, 2], [1, 206, 96, 24, 2]]


In [21]:
# 실제값 정수인코딩
decoder_target = []

for line in lines.tar:
  t = 0
  temp_X = []
  for w in line:
    if t > 0:
      temp_X.append(tar_to_index[w])
    t += 1
  decoder_target.append(temp_X)

In [22]:
print(decoder_target[:5])

[[44, 11, 2], [548, 195, 11, 2], [288, 570, 4, 2], [288, 570, 11, 2], [206, 96, 24, 2]]


- 패딩작업

In [23]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])

In [24]:
max_src_len, max_tar_len

(537, 298)

In [25]:
# 영어는 영어 길이, 한국어는 한국어 길이에 맞춰 패딩
encoder_input = pad_sequences(encoder_input,
                              maxlen = max_src_len,
                              padding = 'post')
decoder_input = pad_sequences(decoder_input,
                              maxlen = max_tar_len,
                              padding = 'post')
decoder_target = pad_sequences(decoder_target,
                               maxlen = max_tar_len,
                               padding = 'post')

- 원핫인코딩
  - 글자단위 번역기므로 워드임베딩 사용하지 않음
  - 입력값도 원핫벡터 사용

In [26]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

### 2) 모델 설계, 훈련

In [27]:
encoder_inputs = Input(shape = (None, src_vocab_size))
encoder_lstm = LSTM(units = 256, return_state = True) # 인코더의 내부 상태를 디코더로 넘겨줌
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# encoder_outputs는 리턴받았지만 필요없으므로 사용하지 않음

encoder_states = [state_h, state_c]
# LSTM은 RNN과 달리 상태가 두개(은닉 상태, 셀 상태)
# 두가지 상태를 모두 디코더로 전달(컨텍스트 벡터)

In [28]:
decoder_inputs = Input(shape = (None, tar_vocab_size))
decoder_lstm = LSTM(units = 256,
                    return_sequences = True,
                    return_state = True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state = encoder_states)
# 디코더의 첫 상태를 인코더의 은닉상태, 셀 상태로 지정

decoder_softmax_layer = Dense(tar_vocab_size,
                              activation = 'softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [29]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [30]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy')

In [31]:
model.fit(x = [encoder_input, decoder_input],
          y = decoder_target,
          batch_size = 64,
          epochs = 200,
          validation_split = .2)

Epoch 1/200
46/46 [==============================] - 41s 205ms/step - loss: 3.0841 - val_loss: 0.4909
Epoch 2/200
46/46 [==============================] - 8s 180ms/step - loss: 0.2392 - val_loss: 0.4833
Epoch 3/200
46/46 [==============================] - 8s 178ms/step - loss: 0.2324 - val_loss: 0.4665
Epoch 4/200
46/46 [==============================] - 8s 180ms/step - loss: 0.2271 - val_loss: 0.4587
Epoch 5/200
46/46 [==============================] - 8s 179ms/step - loss: 0.2147 - val_loss: 0.4622
Epoch 6/200
46/46 [==============================] - 8s 178ms/step - loss: 0.2046 - val_loss: 0.4091
Epoch 7/200
46/46 [==============================] - 8s 181ms/step - loss: 0.1960 - val_loss: 0.3915
Epoch 8/200
46/46 [==============================] - 8s 176ms/step - loss: 0.1881 - val_loss: 0.3840
Epoch 9/200
46/46 [==============================] - 8s 178ms/step - loss: 0.1797 - val_loss: 0.3755
Epoch 10/200
46/46 [==============================] - 8s 177ms/step - loss: 0.1790 - val_l

### 3) 번역기 동작

In [32]:
encoder_model = Model(inputs = encoder_inputs,
                      outputs = encoder_states)

In [33]:
# 이전 시점 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape = (256,))
decoder_state_input_c = Input(shape = (256,))
decoder_states_inputs = [decoder_state_input_h , decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs,
                                                 initial_state = decoder_states_inputs)
# 다음 단어를 예측하기 위해 초기상태를 이전 시점의 상태로 사용
# 이는 뒤 함수 decode_sequence()에 구현

decoder_states = [state_h, state_c]

decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs = [decoder_inputs] + decoder_states_inputs,
                      outputs = [decoder_outputs] + decoder_states)

In [34]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [35]:
def decode_sequence(input_seq) :

  #입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원핫벡터 생성
  target_seq = np.zeros((1, 1, tar_vocab_size))
  target_seq[0, 0, tar_to_index['\t']] = 1.

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 반복
  while not stop_condition:

    # 이전 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측문자를 예측문장에 추가
    decoded_sentence += sampled_char

    # <eos>에 도달하거나 최대 길이를 넘으면 중단
    if (sampled_char == '\n' or len(decoded_sentence) > max_tar_len) :
        stop_condition = True

    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

In [36]:
for i in range(10) :
  seq_index = random.randint(10, 300)
  input_seq = encoder_input[seq_index:seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)

  print('-' * 35)
  print('입력문장: ' , lines.src[seq_index])
  print('정답문장: ' , lines.tar[seq_index][1:len(lines.tar[seq_index]) - 1])
  # '\t' , '\n' 빼고 출력
  print('번역기가 번역한 문장: ' , decoded_sentence[:len(decoded_sentence) -1])
  # '\n'을 빼고 출력

-----------------------------------
입력문장:  Is Tom ill?
정답문장:  톰은 아파?
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Is that OK?
정답문장:  괜찮은 거예요?
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Skip it.
정답문장:  건너뛰어.
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  What fun!
정답문장:  재밌잖아!
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Tom won.
정답문장:  톰이 이겼어.
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Who died?
정답문장:  누가 죽었어?
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Forget Tom.
정답문장:  톰은 잊어버려.
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Take this.
정답문장:  이걸 가져.
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  It works.
정답문장:  되네.
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.
-----------------------------------
입력문장:  Come aboard.
정답문장:  외국으로 와.
번역기가 번역한 문장:  톰은 자기 컴퓨터를 껐어.


## 2. 단어 단위 번역기

### 1) 데이터 전처리
- 전처리 함수 구현

In [8]:
def unicode_to_ascii(s) :
  return ''.join(c for c in unicodedata.normalize('NFD' , s)
  if unicodedata.category(c) != 'Mn')

In [9]:
def preprocess_sentence(sent) :
  # 위에서 구현한 함수를 내부적으로 호출
  # sent = unicode_to_ascii(sent.lower())

  # 단어와 구두점 사이에 공백 만듦
  sent = re.sub(r'([?.!,¿])' , r' \1' , sent)

  # (a-z, A-Z, '.', '?', '!', ',')를 제외하고 전부 공백으로 변환
  sent = re.sub(r'[^a-zA-Z!.?ㄱ-하-ㅣ가-힣]+' , r' ' , sent)

  # sent = re.sub(r'\s+' , ' ' , sent)
  return sent

- 전처리 테스트

In [10]:
en_sent = u'Have you had dinner?'
ko_sent = u'밥 먹었니?'

print(preprocess_sentence(en_sent))
print(preprocess_sentence(ko_sent))

Have you had dinner ?
밥 먹었니 ?


In [11]:
def load_preprocessed_data():
  encoder_input, decoder_input, decoder_target = [], [], []

  with open('kor.txt' , 'r' , encoding = 'utf-8') as lines:
    for i, line in enumerate(lines):

      # source 데이터와 target 데이터 분리
      src_line, tar_line, _ = line.strip().split('\t')

      # source 데이터 전처리
      src_line_input = [ w for w in preprocess_sentence(src_line).split()]

      # target 데이터 전처리
      tar_line = preprocess_sentence(tar_line)
      tar_line_input = [ w for w in ("<sos> " + tar_line).split()]
      tar_line_target = [ w for w in (tar_line + " <sos>").split()]

      encoder_input.append(src_line_input)
      decoder_input.append(tar_line_input)
      decoder_target.append(tar_line_target)


  return encoder_input, decoder_input, decoder_target

In [12]:
sents_en_in, sents_ko_in, sents_ko_out = load_preprocessed_data()

print(sents_en_in[:5])
print(sents_ko_in[:5])
print(sents_ko_out[:5])

[['Go', '.'], ['Hi', '.'], ['Run', '!'], ['Run', '.'], ['Who', '?']]
[['<sos>', '가', '.'], ['<sos>', '안녕', '.'], ['<sos>', '뛰어', '!'], ['<sos>', '뛰어', '.'], ['<sos>', '누구', '?']]
[['가', '.', '<sos>'], ['안녕', '.', '<sos>'], ['뛰어', '!', '<sos>'], ['뛰어', '.', '<sos>'], ['누구', '?', '<sos>']]


- 단어집합 생성, sequence-to-sequence로 변환하는 정수 인코딩

In [13]:
tokenizer_en = Tokenizer(filters = '', lower = False)
tokenizer_en.fit_on_texts(sents_en_in)
encoder_input = tokenizer_en.texts_to_sequences(sents_en_in)

tokenizer_ko = Tokenizer(filters = '', lower = False)
tokenizer_ko.fit_on_texts(sents_ko_in)
tokenizer_ko.fit_on_texts(sents_ko_out)
decoder_input = tokenizer_ko.texts_to_sequences(sents_ko_in)
decoder_target = tokenizer_ko.texts_to_sequences(sents_ko_out)

- 패딩

In [14]:
encoder_input = pad_sequences(encoder_input, padding = 'post')
decoder_input = pad_sequences(decoder_input, padding = 'post')
decoder_target = pad_sequences(decoder_target, padding = 'post')

In [15]:
src_vocab_size = len(tokenizer_en.word_index) + 1
tar_vocab_size = len(tokenizer_ko.word_index) + 1

print('영어 단어 집합의 크기: {:d}, 한국어 단어 집합의 크기: {:d}' .format(src_vocab_size, tar_vocab_size))

영어 단어 집합의 크기: 2751, 한국어 단어 집합의 크기: 5520


- 단어 -> 정수, 정수 -> 단어 딕셔너리 생성

In [16]:
src_to_index = tokenizer_en.word_index
index_to_src = tokenizer_en.index_word # 훈련 후 결과 비교할 때 사용

tar_to_index = tokenizer_ko.word_index # 훈련 후 예측 과정에서 사용
index_to_tar = tokenizer_ko.index_word # 훈련 후 결과 비교할 때 사용

- 적절한 분포를 갖도록 데이터를 섞어줌

In [17]:
encoder_input.shape[0]

3648

In [18]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)

indices

array([2712, 2129,  610, ...,  928,  149, 1200])

In [19]:
# indices를 데이터셋의 순서로 지정해주면 샘플들이 기존순서와 다른 순서로 섞임
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]

In [20]:
encoder_input[2000] , decoder_input[2000] , decoder_target[2000]

(array([  34,   11,   58, 1666,    1,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0], dtype=int32),
 array([   1,   34,  142, 2405,    2,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    

- 훈련데이터의 10%를 테스트 데이터로 분리

In [21]:
n_of_val = encoder_input.shape[0] // 10

n_of_val

364

In [22]:
encoder_input_train = encoder_input[:-n_of_val]
decoder_input_train = decoder_input[:-n_of_val]
decoder_target_train = decoder_target[:-n_of_val]

encoder_input_test = encoder_input[-n_of_val:]
decoder_input_test = decoder_input[-n_of_val:]
decoder_target_test = decoder_target[-n_of_val:]

In [23]:
encoder_input_train.shape, decoder_input_train.shape, decoder_target_train.shape

((3284, 102), (3284, 92), (3284, 92))

In [24]:
encoder_input_test.shape, decoder_input_test.shape, decoder_target_test.shape

((364, 102), (364, 92), (364, 92))

### 2) 기계번역기 만들기

- 임베딩 벡터와 LSTM 은닉상태의 크기를 50으로 고정

In [25]:
latent_dim = 50

- 인코더 설계

In [28]:
encoder_inputs = Input(shape = (None,))
enc_emb = Embedding(src_vocab_size, latent_dim)(encoder_inputs)  # 임베딩 층
enc_masking = Masking(mask_value = 0.0)(enc_emb)  # 패딩 0은 연산에서 제외
encoder_lstm = LSTM(latent_dim, return_state = True)  # 상태값 리턴을 위해 return_state = True
encoder_outputs, state_h, state_c = encoder_lstm(enc_masking) # 은닉상태와 셀 상태를 리턴
encoder_states = [state_h, state_c]  # 인코더의 은닉 상태와 셀 상태를 저장

- 디코더 설계

In [29]:
decoder_inputs = Input(shape = (None,))
dec_emb_layer = Embedding(tar_vocab_size, latent_dim) # 임베딩 층
dec_emb = dec_emb_layer(decoder_inputs) # 패딩 0은 연산에서 제외
dec_masking = Masking(mask_value = 0.0)(dec_emb)

decoder_lstm = LSTM(latent_dim,
                    return_sequences = True,  # 모든 시점에 대해서 단어를 예측하기 위해
                    return_state = True)      # 상태값 리턴을 위해

decoder_outputs, _, _ = decoder_lstm(dec_masking,
                                     initial_state = encoder_states)
# 인코더의 은닉 상태를 초기 은닉상태로 사용

decoder_dense = Dense(tar_vocab_size, activation = 'softmax')
decoder_outputs = decoder_dense(decoder_outputs)
# 모든 시점의 결과에 대해 소프트맥스 함수를 사용해 단어 예측

In [30]:
model = Model([encoder_inputs, decoder_inputs] , decoder_outputs)

In [31]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 50)     137550      input_3[0][0]                    
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     276000      input_4[0][0]                    
______________________________________________________________________________________________

In [32]:
# 원핫인코딩하지 않은 상태로 다중클래스 분류를 하기 위해서 sparse_categorical_crossentropy 사용
model.compile(loss = 'sparse_categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics = ['accuracy'])

In [33]:
%%time

hist = model.fit(x = [encoder_input_train, decoder_input_train],
                 y = decoder_target_train,
                 validation_data = ([encoder_input_test, decoder_input_test],
                                    decoder_target_test),
                 batch_size = 128,
                 epochs = 50)

Epoch 1/50
26/26 [==============================] - 16s 136ms/step - loss: 7.6651 - accuracy: 0.7983 - val_loss: 4.5591 - val_accuracy: 0.9322
Epoch 2/50
26/26 [==============================] - 1s 49ms/step - loss: 3.8332 - accuracy: 0.9332 - val_loss: 1.9784 - val_accuracy: 0.9322
Epoch 3/50
26/26 [==============================] - 1s 49ms/step - loss: 1.5984 - accuracy: 0.9323 - val_loss: 0.8533 - val_accuracy: 0.9322
Epoch 4/50
26/26 [==============================] - 1s 50ms/step - loss: 0.7547 - accuracy: 0.9328 - val_loss: 0.5792 - val_accuracy: 0.9322
Epoch 5/50
26/26 [==============================] - 1s 50ms/step - loss: 0.5535 - accuracy: 0.9319 - val_loss: 0.4899 - val_accuracy: 0.9322
Epoch 6/50
26/26 [==============================] - 1s 51ms/step - loss: 0.4840 - accuracy: 0.9313 - val_loss: 0.4500 - val_accuracy: 0.9322
Epoch 7/50
26/26 [==============================] - 1s 50ms/step - loss: 0.4313 - accuracy: 0.9326 - val_loss: 0.4305 - val_accuracy: 0.9325
Epoch 8/50


### 3) 기계번역기 동작시키기

- 인코더 설계

In [34]:
encoder_model = Model(encoder_inputs, encoder_states)

In [35]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding_2 (Embedding)      (None, None, 50)          137550    
_________________________________________________________________
masking_2 (Masking)          (None, None, 50)          0         
_________________________________________________________________
lstm_2 (LSTM)                [(None, 50), (None, 50),  20200     
Total params: 157,750
Trainable params: 157,750
Non-trainable params: 0
_________________________________________________________________


- 디코더 설계

In [36]:
# 이전 시점의 상태를 보관할 텐서
decoder_state_input_h = Input(shape = (latent_dim, ))
decoder_state_input_c = Input(shape = (latent_dim, ))
decoder_states_inputs = [decoder_state_input_h , decoder_state_input_c]

# 훈련 때 사용했던 임베딩 층 재사용
dec_emb2 = dec_emb_layer(decoder_inputs)

# 다음 단어 예측을 위해 예전 시점의 상태를 현 시점의 초기 상태로 사용
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2,
                                                    initial_state = decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]

# 모든 시점에 대해서 단어 예측
decoder_outputs2 = decoder_dense(decoder_outputs2)

- 디코더 정의

In [37]:
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

In [38]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 50)     276000      input_4[0][0]                    
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 50)]         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            [(None, 50)]         0                                            
____________________________________________________________________________________________

- 동작을 위한 decode_sequence 함수 구현

In [40]:
def decode_sequence(input_seq) :

  # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <sos>에 해당하는 정수 생성
  target_seq = np.zeros((1,1))
  target_seq[0, 0] = tar_to_index['<sos>']

  stop_condition = False
  decoded_sentence = ''

  # stop_condition이 True가 될 때까지 루ㅠㅡ 반복

  while not stop_condition:
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 에측 결과를 단어로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 단어를 예측 문장에 추가
    decoded_sentence += ' '+sampled_char

    # <eos>에 도달하거나 정해진 길이를 넘으면 중단
    if (sampled_char == '<eos>' or len(decoded_sentence) > 50):
      stop_condition = True
    
    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = sampled_token_index

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence

- 결과 확인을 위한 함수 생성

In [41]:
# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2seq(input_seq):
  temp = ''
  for i in input_seq:
    if (i != 0):
      temp = temp + index_to_src[i] + ' '
  return temp

In [57]:
# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq2tar(input_seq):
  temp = ''
  for i in input_seq:
    if ((i!=0 and i!=tar_to_index['<sos>'])):
      temp = temp + index_to_tar[i] + ' '
  return temp

- 임의로 선택한 인덱스의 샘플 결과를 출력

In [58]:
for seq_index in [3, 50, 100, 300, 1001] :
  input_seq = encoder_input_train[seq_index: seq_index + 1]
  decoded_sentence = decode_sequence(input_seq)


  print("원문: " , seq2seq(encoder_input_train[seq_index]))
  print("번역문: " , seq2tar(decoder_input_train[seq_index]))
  print("예측문: " , decoded_sentence[:-5])
  print('\n')

원문:  Her dress attracted everyone s attention at the party . 
번역문:  그 여자가 파티에 입고 온 드레스가 사람들의 눈길을 끌었지 . 
예측문:   톰은 사람은 프랑스어를 안 않아 . <sos> 있어 . <sos> ? <sos> ? 


원문:  I can t sleep . 
번역문:  잠이 와 . 
예측문:   톰은 사람은 프랑스어를 안 수 있어 . <sos> 있어 . <sos> ? <sos> 


원문:  I looked for the key under the welcome mat and in the nearby flower pot . 
번역문:  나는 열쇠를 찾기 위해 현관 매트 아래와 근처 화분 안을 뒤졌다 . 
예측문:   톰은 사람은 프랑스어를 안 않아 . <sos> 있어 . <sos> ? <sos> ? 


원문:  Tom drove . 
번역문:  톰이 운전했어 . 
예측문:   톰은 사람은 프랑스어를 안 수 있어 . <sos> 있어 . <sos> ? <sos> 


원문:  Her faith in God is unshaken . 
번역문:  그녀의 신앙심은 굳건하다 . 
예측문:   톰은 사람은 프랑스어를 안 않아 . <sos> 있어 . <sos> ? <sos> 




- 느낀점
  - LSTM으로 번역을 하기 매우 어렵다
  - 해보았다는 데 의의를 둔다